### RapidFire AI Tutorial Use Case: SFT for Customer Support Q&A Chatbot

In [1]:
from rapidfireai import Experiment
from rapidfireai.automl import List, RFGridSearch, RFModelConfig, RFLoraConfig, RFSFTConfig

### Load Dataset and Specify Train and Eval Partitions

In [2]:
from datasets import load_dataset

dataset=load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")

# Select a subset of the dataset for demo purposes
train_dataset=dataset["train"].select(range(50))
eval_dataset=dataset["train"].select(range(5000,5020))
train_dataset=train_dataset.shuffle(seed=42)
eval_dataset=eval_dataset.shuffle(seed=42)

### Define Data Processing Function

In [3]:
def sample_formatting_function(row):
    """Function to preprocess each example from dataset"""
    # Special tokens for formatting
    SYSTEM_PROMPT = "You are a helpful and friendly customer support assistant. Please answer the user's query to the best of your ability."
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": row["instruction"]},
            
        ],
        "completion": [
            {"role": "assistant", "content": row["response"]}
        ]
    }

### Initialize Experiment

In [4]:
# Every experiment instance must be uniquely named
experiment = Experiment(experiment_name="exp2-chatqa")

2025/09/16 19:19:22 INFO mlflow.tracking.fluent: Experiment with name 'exp2-chatqa_19' does not exist. Creating a new experiment.


Task Training and Validation cancelled
No active MLflow run to clear
The previously running experiment exp2-chatqa_18 was forcibly ended. Created a new experiment with name 'exp2-chatqa_19' with Experiment ID: 51 and MLFlow Experiment ID: 51 saved at /home/palebluedot/rapidfireai/tutorial_notebooks/rapidfire_experiments/exp2-chatqa_19


### Define Custom Eval Metrics Function

In [5]:
def sample_compute_metrics(eval_preds):  
    """Optional function to compute eval metrics based on predictions and labels"""
    predictions, labels = eval_preds

    # Standard text-based eval metrics: Rouge and BLEU
    import evaluate
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")

    rouge_output = rouge.compute(predictions=predictions, references=labels, use_stemmer=True)
    rouge_l = rouge_output["rougeL"]
    bleu_output = bleu.compute(predictions=predictions, references=labels)
    bleu_score = bleu_output["bleu"]

    return {
        "rougeL": round(rouge_l, 4),
        "bleu": round(bleu_score, 4),
    }

### Define Multi-Config Knobs for Model, LoRA, and SFT Trainer using RapidFire AI Wrapper APIs

In [6]:
# 2 LoRA PEFT configs with different adapter capacities
peft_configs = List([
    RFLoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["q_proj", "v_proj"],
        bias="none"
    ),
    RFLoraConfig(
            r=128,
            lora_alpha=256,
            lora_dropout=0.05,
            target_modules=["q_proj","k_proj", "v_proj","o_proj"],
            bias="none"
    )
])

# 2 base models x 2 peft configs = 4 combinations in total
config_set = List([
    # RFModelConfig(
    #     model_name="meta-llama/Llama-3.1-8B-Instruct",
    #     peft_config=peft_configs,
    #     training_args=RFSFTConfig(
    #         learning_rate=2e-4,
    #         lr_scheduler_type="linear",
    #         per_device_train_batch_size=4,
    #         per_device_eval_batch_size=8,
    #         num_train_epochs=2,
    #         gradient_accumulation_steps=4,
    #         logging_steps=5,
    #         eval_strategy="steps",
    #         eval_steps=25,
    #         fp16=True,
    #         save_strategy="epoch"
    #     ),
    #     model_type="causal_lm",
    #     model_kwargs={"device_map": "auto", "torch_dtype": "auto","use_cache":False},
    #     formatting_func = sample_formatting_function,
    #     compute_metrics = sample_compute_metrics,
    #     generation_config = { # This is for text based evaluation/prediction for causal_lm models
    #         "max_new_tokens": 256,
    #         "temperature": 0.6,
    #         "top_p": 0.9,
    #         "top_k": 40,
    #         "repetition_penalty": 1.18,
    #     }
    # ),
    RFModelConfig(
        model_name="meta-llama/Llama-3.1-8B-Instruct",
        peft_config=peft_configs,
        training_args=RFSFTConfig(
            learning_rate=2e-5,
            lr_scheduler_type="linear",
            per_device_train_batch_size=2,
            per_device_eval_batch_size=2,
            num_train_epochs=1,
            optim="adamw_torch",
            # gradient_accumulation_steps=4,
            logging_steps=5,
            eval_strategy="steps",
            eval_steps=25,
            fp16=False,
            bf16=True,
            save_strategy="no",
            fsdp="full_shard auto_wrap",
            fsdp_config={"backward_prefetch": "backward_pre","forward_prefetch": True,"use_orig_params": False,  "cpu_ram_efficient_loading": False,"sync_module_states": True,"min_num_params": 1000000,"limit_all_gathers": True}
        ),
        model_type="causal_lm",
        model_kwargs={"device_map": None, "torch_dtype": "bfloat16","use_cache":False},
        formatting_func = sample_formatting_function,
        compute_metrics = sample_compute_metrics,
        generation_config = { # This is for text based evaluation/prediction for causal_lm models
            "max_new_tokens": 256,
            "temperature": 0.6,
            "top_p": 0.9,
            "top_k": 40,
            "repetition_penalty": 1.18,
        },
        num_gpus=2
    )
])


#### Define Model Creation Function for All Model Types Across Configs

In [7]:

def sample_create_model(model_config): 
     """Function to create model object for any given config; must return tuple of (model, tokenizer)"""
     from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, BitsAndBytesConfig
     import torch

     model_name = model_config["model_name"]
     model_type = model_config["model_type"]
     model_kwargs = model_config["model_kwargs"]
     bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,     # compute dtype
        bnb_4bit_quant_storage=torch.bfloat16,     # <-- critical for FSDP flatten
     )
     if model_type == "causal_lm":
          model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)
     elif model_type == "seq2seq_lm":
          model = AutoModelForSeq2SeqLM.from_pretrained(model_name, **model_kwargs)
     elif model_type == "masked_lm":
          model = AutoModelForMaskedLM.from_pretrained(model_name, **model_kwargs)
     elif model_type == "custom":
          # Handle custom model loading logic, e.g., loading your own checkpoints
          # model = ... 
          pass
     else:
          # Default to causal LM
          model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs, quantization_config=bnb_config)
      
     tokenizer = AutoTokenizer.from_pretrained(model_name)
      
     return (model,tokenizer)


#### Generate Config Group

In [8]:
# Simple grid search across all sets of config knob values = 4 combinations in total
config_group = RFGridSearch(
    configs=config_set,
    trainer_type="SFT"
)

### Run Multi-Config Training

In [9]:
# Launch training of all configs in the config_group with swap granularity of 4 chunks
experiment.run_fit(config_group, sample_create_model, train_dataset, eval_dataset,num_chunks=2, num_gpus=4,seed=42)

Started 4 worker processes successfully
Created workers


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.


params in peft state dict before collecting 2034828288
params in peft state dict before collecting params in peft state dict before collecting2034828288
 2034828288
params in peft state dict before collecting 2034828288


/home/palebluedot/rapidfireai/oss_venv/lib/python3.12/site-packages/torch/distributed/fsdp/fully_sharded_data_parallel.py:678: FutureWarning: FSDP.state_dict_type() and FSDP.set_state_dict_type() are being deprecated. Please use APIs, get_state_dict() and set_state_dict(), which can support different parallelisms, FSDP1, FSDP2, DDP. API doc: https://pytorch.org/docs/stable/distributed.checkpoint.html#torch.distributed.checkpoint.state_dict.get_state_dict .Tutorial: https://pytorch.org/tutorials/recipes/distributed_checkpoint_recipe.html .
  warnings.warn(
/home/palebluedot/rapidfireai/oss_venv/lib/python3.12/site-packages/torch/distributed/fsdp/fully_sharded_data_parallel.py:678: FutureWarning: FSDP.state_dict_type() and FSDP.set_state_dict_type() are being deprecated. Please use APIs, get_state_dict() and set_state_dict(), which can support different parallelisms, FSDP1, FSDP2, DDP. API doc: https://pytorch.org/docs/stable/distributed.checkpoint.html#torch.distributed.checkpoint.state

total peft params 2034828288
params in peft state dict after collecting on worker  1 :  0
total peft params 2034828288
params in peft state dict after collecting on worker  2 :  0
total peft params 2034828288
params in peft state dict after collecting on worker  3 :  0
total peft params 2034828288
params in peft state dict after collecting on worker  0 :  109051904


[rank1]:[W916 19:22:46.593731845 PyInterpreter.cpp:263] Warning: Deallocating Tensor that still has live PyObject references.  This probably happened because you took out a weak reference to Tensor and didn't call _fix_weakref() after dereferencing it.  Subsequent accesses to this tensor via the PyObject will now fail. (function decref)
[rank1]:[W916 19:22:46.602374422 PyInterpreter.cpp:263] Warning: Deallocating Tensor that still has live PyObject references.  This probably happened because you took out a weak reference to Tensor and didn't call _fix_weakref() after dereferencing it.  Subsequent accesses to this tensor via the PyObject will now fail. (function decref)
[rank1]:[W916 19:22:46.609163656 PyInterpreter.cpp:263] Warning: Deallocating Tensor that still has live PyObject references.  This probably happened because you took out a weak reference to Tensor and didn't call _fix_weakref() after dereferencing it.  Subsequent accesses to this tensor via the PyObject will now fail. (

KeyboardInterrupt: 

### End Current Experiment

In [ ]:
experiment.end()